# 1. without expanding to USA

In [ ]:
from pyinaturalist import get_observations, get_taxa
import time

NC_PLACE_ID = 30
USA_PLACE_ID = 1

# IDs found via iNaturalist taxonomy search
classes = {
    0:  {'name': 'Venomous Spiders',   'taxon_ids': [47370, 48140]}, # Latrodectus, Sicariidae
    1:  {'name': 'Ticks',              'taxon_ids': [51673]},        # Ixodidae 
    2:  {'name': 'Mosquitoes',         'taxon_ids': [52134]},        # Culicidae
    3:  {'name': 'Stinging Wasps',     'taxon_ids': [52747]},        # Vespidae
    4:  {'name': 'Bees',               'taxon_ids': [47221]},        # Apidae
    5:  {'name': 'Fire Ants',          'taxon_ids': [67597]},        # Genus Solenopsis
    6:  {'name': 'Assassin Bugs',      'taxon_ids': [48959]},        # Reduviidae
    7:  {'name': 'Venomous Caterpillars', 'taxon_ids': [84186, 84165]}, # Megalopygidae, Limacodidae
    8:  {'name': 'Blister Beetles',    'taxon_ids': [59510]},        # Meloidae 
    9:  {'name': 'Scorpions',          'taxon_ids': [52572]},        # Vaejovidae 
    10: {'name': 'Horse/Deer Flies',   'taxon_ids': [47821]},        # Tabanidae 
    11: {'name': 'Centipedes',         'taxon_ids': [49556]}         # Order Chilopoda
}

# List of IDs that require USA-wide search due to low sample size in NC
EXPAND_TO_USA = [
    #48140, # Sicariidae (Subset of Venomous Spiders)
    #52572, # Vaejovidae (Scorpions)
    #59510, # Meloidae (Blister Beetles)
    #47821, # Tabanidae (Horse/Deer Flies),
    #84186, # Megalopygidae (Subset of Venoomous Caterpillars)
]

def get_counts(classes):
    print(f"{'Class Group':<22} {'Specific Taxon Name':<25} {'ID':<8} {'Loc':<5} {'Count':<8}")
    print("-" * 75)

    total_pipeline_images = 0

    for class_id, info in classes.items():
        class_total = 0
        
        for taxon_id in info['taxon_ids']:
            
            if taxon_id in EXPAND_TO_USA:
                current_place = USA_PLACE_ID
                loc_name = "USA"
            else:
                current_place = NC_PLACE_ID
                loc_name = "NC"

            params = {
                'taxon_id': taxon_id,
                'place_id': current_place,
                'quality_grade': "research",
                'count_only': True
            }

            # Special Logic: Add Life Stage filter ONLY for Caterpillars
            if class_id == 7:
                params['term_id'] = 1       # Annotation Attribute: Life Stage
                params['term_value_id'] = 6 # Annotation Value: Larva

            try:
                taxon_info = get_taxa(taxon_id=taxon_id)
                if taxon_info['results']:
                    taxon_name = taxon_info['results'][0].get('preferred_common_name', taxon_info['results'][0]['name'])
                else:
                    taxon_name = "Unknown Name"

                response = get_observations(**params)
                count = response['total_results']
                
                print(f"{info['name']:<22} {taxon_name[:24]:<25} {str(taxon_id):<8} {loc_name:<5} {count:<8}")
                
                class_total += count
                time.sleep(0.3)
                
            except Exception as e:
                print(f"Error processing ID {taxon_id}: {e}")

        total_pipeline_images += class_total
        print("-" * 75)

    print(f"\nTotal Potential Images in Pipeline: {total_pipeline_images}")

if __name__ == "__main__":
    get_counts(classes)

In [ ]:
# IDs found via iNaturalist taxonomy search
classes = {
    0:  {'name': 'Venomous Spiders',   'taxon_ids': [47370, 48140]}, # Latrodectus (not Theridiidae as this includes non harmful), Sicariidae
    1:  {'name': 'Ticks',              'taxon_ids': [51673]},        # Ixodidae 
    2:  {'name': 'Mosquitoes',         'taxon_ids': [52134]},        # Culicidae
    3:  {'name': 'Stinging Wasps',     'taxon_ids': [52747]},        # Vespidae
    4:  {'name': 'Bees',               'taxon_ids': [47221]},        # Apidae
    5:  {'name': 'Fire Ants',          'taxon_ids': [67597]},        # Genus Solenopsis
    6:  {'name': 'Assassin Bugs',      'taxon_ids': [48959]},        # Reduviidae
    7:  {'name': 'Venomous Caterpillers.',     'taxon_ids': [84186, 84165]}, # Megalopygidae, Limacodidae  # use annotation
    8:  {'name': 'Blister Beetles',    'taxon_ids': [59510]},        # Meloidae 
    9:  {'name': 'Scorpions',          'taxon_ids': [52572]},        # Vaejovidae 
    10: {'name': 'Horse/Deer Flies',   'taxon_ids': [47821]},        # Tabanidae 
    11: {'name': 'Centipedes',         'taxon_ids': [49556]}         # Order Chilopoda
}

In [4]:
get_counts(classes, 30) # NC

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     1188
1    Ticks                1292
2    Mosquitoes           1486
3    Stinging Wasps       16308
4    Bees                 38268
5    Fire Ants            1536
6    Assassin Bugs        8212
7    Venomous Caterpillers. 6183
8    Blister Beetles      755
9    Scorpions            26
10   Horse/Deer Flies     652
11   Centipedes           2158


In [5]:
get_counts(classes, 1) # USA

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     48179
1    Ticks                47878
2    Mosquitoes           30436
3    Stinging Wasps       416293
4    Bees                 1173495
5    Fire Ants            23986
6    Assassin Bugs        151267
7    Venomous Caterpillers. 78986
8    Blister Beetles      36288
9    Scorpions            26090
10   Horse/Deer Flies     15505
11   Centipedes           43154


In [35]:
get_counts(classes, None) # world

ID   Class Name           Total Imgs (NC)
----------------------------------------
0    Venomous Spiders     77358
1    Ticks                65751
2    Mosquitoes           55342
3    Stinging Wasps       773799
4    Bees                 2036818
5    Fire Ants            29182
6    Assassin Bugs        232046
7    Venomous Cats.       36864
8    Blister Beetles      89148
9    Scorpions            28889
10   Horse/Deer Flies     35505
11   Centipedes           92593


# Fixed

In [ ]:
from pyinaturalist import get_observations, get_taxa
import time

NC_PLACE_ID = 30
USA_PLACE_ID = 1

# IDs found via iNaturalist taxonomy search
classes = {
    0:  {'name': 'Venomous Spiders',   'taxon_ids': [47370, 48140]}, # Latrodectus, Sicariidae
    1:  {'name': 'Ticks',              'taxon_ids': [51673]},        # Ixodidae 
    2:  {'name': 'Mosquitoes',         'taxon_ids': [52134]},        # Culicidae
    3:  {'name': 'Stinging Wasps',     'taxon_ids': [52747]},        # Vespidae
    4:  {'name': 'Bees',               'taxon_ids': [47221]},        # Apidae
    5:  {'name': 'Fire Ants',          'taxon_ids': [67597]},        # Genus Solenopsis
    6:  {'name': 'Assassin Bugs',      'taxon_ids': [48959]},        # Reduviidae
    7:  {'name': 'Venomous Caterpillars', 'taxon_ids': [84186, 84165]}, # Megalopygidae, Limacodidae
    8:  {'name': 'Blister Beetles',    'taxon_ids': [59510]},        # Meloidae 
    9:  {'name': 'Scorpions',          'taxon_ids': [52572]},        # Vaejovidae 
    10: {'name': 'Horse/Deer Flies',   'taxon_ids': [47821]},        # Tabanidae 
    11: {'name': 'Centipedes',         'taxon_ids': [49556]}         # Order Chilopoda
}

# List of IDs that require USA-wide search due to low sample size in NC
EXPAND_TO_USA = [
    48140, # Sicariidae (Subset of Venomous Spiders)
    52572, # Vaejovidae (Scorpions)
    59510, # Meloidae (Blister Beetles)
    47821, # Tabanidae (Horse/Deer Flies),
    84186, # Megalopygidae (Subset of Venoomous Caterpillars)
]

def get_counts(classes):
    print(f"{'Class Group':<22} {'Specific Taxon Name':<25} {'ID':<8} {'Loc':<5} {'Count':<8}")
    print("-" * 75)

    total_pipeline_images = 0

    for class_id, info in classes.items():
        class_total = 0
        
        for taxon_id in info['taxon_ids']:
            
            if taxon_id in EXPAND_TO_USA:
                current_place = USA_PLACE_ID
                loc_name = "USA"
            else:
                current_place = NC_PLACE_ID
                loc_name = "NC"

            params = {
                'taxon_id': taxon_id,
                'place_id': current_place,
                'quality_grade': "research",
                'count_only': True
            }

            # Special Logic: Add Life Stage filter ONLY for Caterpillars
            if class_id == 7:
                params['term_id'] = 1       # Annotation Attribute: Life Stage
                params['term_value_id'] = 6 # Annotation Value: Larva

            try:
                taxon_info = get_taxa(taxon_id=taxon_id)
                if taxon_info['results']:
                    taxon_name = taxon_info['results'][0].get('preferred_common_name', taxon_info['results'][0]['name'])
                else:
                    taxon_name = "Unknown Name"

                response = get_observations(**params)
                count = response['total_results']
                
                print(f"{info['name']:<22} {taxon_name[:24]:<25} {str(taxon_id):<8} {loc_name:<5} {count:<8}")
                
                class_total += count
                time.sleep(0.3)
                
            except Exception as e:
                print(f"Error processing ID {taxon_id}: {e}")

        total_pipeline_images += class_total
        print("-" * 75)

    print(f"\nTotal Potential Images in Pipeline: {total_pipeline_images}")

if __name__ == "__main__":
    get_counts(classes)

Class Group            Specific Taxon Name       ID       Loc   Count   
---------------------------------------------------------------------------
Venomous Spiders       Widow Spiders             47370    NC    1159    
Venomous Spiders       Sixeyed Sicariid Spiders  48140    USA   7109    
---------------------------------------------------------------------------
Ticks                  Hardbacked Ticks          51673    NC    1292    
---------------------------------------------------------------------------
Mosquitoes             Mosquitoes                52134    NC    1486    
---------------------------------------------------------------------------
Stinging Wasps         Hornets, Paper Wasps, Po  52747    NC    16308   
---------------------------------------------------------------------------
Bees                   Honey Bees, Bumble Bees,  47221    NC    38268   
---------------------------------------------------------------------------
Fire Ants              Solenopsis